In [1]:
import itertools
import sys

class CFGParser:
    def __init__(self):
        self.cfg_grammar = {}
        self.cnf_grammar = {}
        self.terminals = set()
        self.non_terminals = set()
        self.start_symbols = set()

    def input_grammar(self):
        print("Enter the context-free grammar:")
        num_productions = int(input("Enter the number of productions: "))
        print("For each production, enter it in the form 'A -> BC | a B a'.")
        print("Separate multiple productions for the same non-terminal with '|'.")
        print("Use 'e' to represent epsilon.")
        for _ in range(num_productions):
            productions_input = input("Productions: ")
            left, right = productions_input.split(' -> ')
            self.non_terminals.add(left)
            productions = right.split(' | ')
            for production in productions:
                if production != 'e':
                    symbols = production.split()
                    for symbol in symbols:
                        if symbol.islower() or symbol.isdigit():
                            self.terminals.add(symbol)
                else:
                    symbols = ['e']
                    print("Using 'e' to represent epsilon.")
                if left in self.cfg_grammar:
                    self.cfg_grammar[left].append(symbols)
                else:
                    self.cfg_grammar[left] = [symbols]
                if left not in self.start_symbols:
                    self.start_symbols.add(left)

    def convert_to_cnf(self):
        # Step 1: Remove epsilon productions
        self._remove_epsilon()

        # Step 2: Convert to CNF
        for nt in self.cfg_grammar:
            for production in self.cfg_grammar[nt]:
                while len(production) > 2:
                    new_nt = self._generate_new_non_terminal()
                    self.non_terminals.add(new_nt)
                    self.cnf_grammar[new_nt] = [production[:2]]
                    production = [new_nt] + [production[2:]]  # Corrected line
                if nt in self.cnf_grammar:
                    self.cnf_grammar[nt].append(production)
                else:
                    self.cnf_grammar[nt] = [production]

    def _generate_new_non_terminal(self):
        # Generate a new non-terminal symbol for CNF conversion
        for i in range(1, len(self.non_terminals) + 2):
            new_nt = 'X' * i
            if new_nt not in self.non_terminals:
                return new_nt

    def _remove_epsilon(self):
        epsilon_producing = set()
        # Find epsilon producing non-terminals
        while True:
            prev_size = len(epsilon_producing)
            for nt, productions in self.cfg_grammar.items():
                for production in productions:
                    if all(symbol in epsilon_producing or symbol == 'e' for symbol in production):
                        epsilon_producing.add(nt)
            if len(epsilon_producing) == prev_size:
                break

        # Remove epsilon productions
        for nt, productions in self.cfg_grammar.items():
            new_productions = []
            for production in productions:
                if all(symbol in epsilon_producing or symbol == 'e' for symbol in production):
                    # Skip epsilon productions
                    continue
                for symbols in itertools.product(*[[symbol, ''] for symbol in production]):
                    if ''.join(symbols):
                        new_productions.append(''.join(symbols))
            self.cfg_grammar[nt] = new_productions

        # Update start symbols
        start_symbols_with_epsilon = self.start_symbols.intersection(epsilon_producing)
        self.start_symbols -= start_symbols_with_epsilon
        if self.start_symbols:
            new_start_symbol = self._generate_new_non_terminal()
            self.non_terminals.add(new_start_symbol)
            self.cnf_grammar[new_start_symbol] = [['e']]  # New start symbol derives epsilon
            for start_symbol in start_symbols_with_epsilon:
                self.cnf_grammar[new_start_symbol].append([start_symbol])
            self.start_symbols.add(new_start_symbol)
        else:
            self.start_symbols.add('S')  # If no start symbols remain, add a new one
            self.cnf_grammar['S'] = [['e']]

    def parse(self, input_str):
        sys.setrecursionlimit(10000)  # Adjust recursion limit
        for start_symbol in self.start_symbols:
            if self._parse(input_str, start_symbol, depth=0):
                return True
        return False

    def _parse(self, input_str, symbol, depth):
        if depth > 1000:  # Set maximum depth
            return False
        if symbol in self.terminals:
            if not input_str:
                return False
            return input_str == symbol
        for production in self.cnf_grammar.get(symbol, []):
            if self._try_production(input_str, production, depth + 1):
                return True
        return False

    def _try_production(self, input_str, production, depth):
        if not input_str and 'e' in production:
            return True
        remaining_input = input_str
        for symbol in production:
            if symbol in self.terminals:
                if not remaining_input or remaining_input[0] != symbol:
                    return False
                remaining_input = remaining_input[1:]
            elif not self._parse(remaining_input, symbol, depth):
                return False
        return True

# Example usage:
print("Welcome to the Context-Free Grammar Parser!")
parser = CFGParser()
parser.input_grammar()
print("\nGrammar has been successfully input.")
print("Terminals:", parser.terminals)
print("Non-terminals:", parser.non_terminals)
parser.convert_to_cnf()
print("\nGrammar converted to CNF.")
print("CNF Grammar:")
for nt, productions in parser.cnf_grammar.items():
    for production in productions:
        print(nt, "->", " ".join(production))
input_str = input("\nEnter the input string to parse: ")
if parser.parse(input_str):
    print("Parse successful!")
else:
    print("Parse failed.")


Welcome to the Context-Free Grammar Parser!
Enter the context-free grammar:
Enter the number of productions: 3
For each production, enter it in the form 'A -> BC | a B a'.
Separate multiple productions for the same non-terminal with '|'.
Use 'e' to represent epsilon.
Productions: S -> 1 A 0
Productions: A -> 1 B 0
Productions: B -> 1 | e
Using 'e' to represent epsilon.

Grammar has been successfully input.
Terminals: {'1', '0'}
Non-terminals: {'S', 'B', 'A'}

Grammar converted to CNF.
CNF Grammar:
X -> e
X -> B
XX -> 1 A
S -> XX 0
S -> 1 A
S -> 1 0
S -> 1
S -> A 0
S -> A
S -> 0
XXX -> 1 B
A -> XXX 0
A -> 1 B
A -> 1 0
A -> 1
A -> B 0
A -> B
A -> 0
B -> 1

Enter the input string to parse: 11100
Parse successful!
